# Исследование надежности заемщиков

## Загрузка данных

In [1]:
# импортируем библиотеку pandas
import pandas as pd

In [7]:
# прочитаем csv-файл
data = pd.read_csv('C:/Users/lazarevnv/OneDrive/1_Обучение/yandex_practicum/Предобработка данных/data.csv')

In [8]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [10]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца income_type

In [11]:
income_type_tolist = data['income_type'].unique().tolist()
income_type_tolist

['сотрудник',
 'пенсионер',
 'компаньон',
 'госслужащий',
 'безработный',
 'предприниматель',
 'студент',
 'в декрете']

In [12]:
data['total_income'].isna().sum()

2174

In [13]:
for incom_type in income_type_tolist:
    data.loc[(data['income_type'] == incom_type) & (data['total_income'].isna()), 'total_income'] = \
    data['total_income'].loc[data['income_type'] == incom_type].median()

In [14]:
data['total_income'].isna().sum()

0

### Обработка аномальных значений

In [15]:
# Скорректируем отрицательные значения в поле days_employed
data['days_employed'] = abs(data['days_employed'])

Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях

In [16]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Выведем перечень уникальных значений столбца children

In [17]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В поле children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data

In [18]:
conditions = (data['children'] != 20) & (data['children'] != -1)
data = data[conditions]

In [19]:
# проверим
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type

In [20]:
for i in income_type_tolist:
    data.loc[(data['income_type'] == i) & (data['days_employed'].isna()), 'days_employed' ] =\
    data[(data['income_type'] == i)]['days_employed'].median()

In [21]:
# Убедимся, что все пропуски заполнены
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце total_income на целочисленный

In [22]:
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

In [23]:
# Посмотрим количество строк-дубликатов в данных
data.duplicated().sum()

54

In [ ]:
# удалим дубликаты
data = data.drop_duplicates()

Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [24]:
data['education'] = data['education'].str.lower()

In [25]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме data столбец total_income_category с категориями:

0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.

In [26]:
# создадим функцию categorize_income()
def categorize_income(value):
    if 0 <= value <= 30000:
        return 'E'
    elif 30001 <= value <= 50000:
        return 'D'
    elif 50001 <= value <= 200000:
        return 'C'
    elif 200001 <= value <= 1000000:
        return 'B'
    elif 1000001 <= value:
        return 'A'

In [27]:
# применим categorize_income через метод apply
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца purpose

In [28]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'

In [29]:
auto_list = [
 'приобретение автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля',
 'автомобили', 'сделка с подержанным автомобилем', 'автомобиль', 'свой автомобиль',
 'сделка с автомобилем', 'на покупку автомобиля'
]

housing_list = ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'покупка недвижимости',
'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости',
 'недвижимость', 'строительство недвижимости', 'операции с коммерческой недвижимостью', 'строительство жилой недвижимости',
 'жилье', 'операции со своей недвижимостью','покупка своего жилья', 'операции с недвижимостью', 'покупка жилья для сдачи',
'ремонт жилью']

merried_list = ['сыграть свадьбу', 'на проведение свадьбы','свадьба']

education_list = ['дополнительное образование', 'образование', 'заняться образованием', 
                  'получение образования', 'получение дополнительного образования', 
                  'получение высшего образования','профильное образование', 
                  'высшее образование', 'заняться высшим образованием']

In [30]:
# создадим функцию categorize_purpose()
def categorize_purpose(line):
    if line in auto_list:
        return 'операции с автомобилем'
    elif line in housing_list:
        return 'операции с недвижимостью'
    elif line in merried_list:
        return 'проведение свадьбы'
    elif line in education_list:
        return 'получение образования'

In [31]:
# примените функцию методом apply()
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [32]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## Исследуем данные

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [33]:
data.pivot_table(index='children', 
                 columns=None, 
                 values='debt', 
                 aggfunc=['count', 'sum', 'mean']).round(3)

,count,sum,mean
,debt,debt,debt
children,,,
0,14149,1063,0.075
1,4818,444,0.092
2,2055,194,0.094
3,330,27,0.082
4,41,4,0.098
5,9,0,0.000


In [34]:
data_grouped = data.groupby('children').agg({'debt':['count', 'sum']})
data_grouped['share_of_debtors_%'] = data_grouped[('debt',   'sum')] / data_grouped[('debt', 'count')] * 100
data_grouped

debt       share_of_debtors_%
          count   sum                   
children                                
0         14149  1063           7.512898
1          4818   444           9.215442
2          2055   194           9.440389
3           330    27           8.181818
4            41     4           9.756098
5             9     0           0.000000

**Вывод:** Значимой зависимости не выявлено. Отклонения не велики, тенденция не проматривается. Можно сделать заключение что зависимости межде колисеством детей и возвратом кредита в срок нет.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
data.pivot_table(index='family_status', columns=None, values='debt', aggfunc=['count', 'sum', 'mean']).round(3)

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2799,273,0.098
в разводе,1189,84,0.071
вдовец / вдова,952,63,0.066
гражданский брак,4160,385,0.093
женат / замужем,12302,927,0.075


**Вывод:** Прослеживается тенденция повышенного % должников среди неженатых заёмщиков и состоящих в гражданском браке. Т.е. можно отметить наличие влияния на возврат кредита в срок.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
data.pivot_table(index='total_income_category', columns=None, values='debt', aggfunc=['count', 'sum', 'mean']).round(3)

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080
B,5014,354,0.071
C,15992,1353,0.085
D,349,21,0.060
E,22,2,0.091


**Вывод:**  В качестве вывода можно отметить две крайние позиции доли должников. Наибольший % должников (9.1%) среди наименее обеспеченной группы "Е". А наименьшая доля должников (6.0%) в группе "D" - второй по обеспеченности с низу. Прямой зависимости между уровнем дохода и возвратом кредита в срок нет. Но есть зависимость от принадлежности к группе дохода.

#### Как разные цели кредита влияют на его возврат в срок?

In [37]:
data.pivot_table(index='purpose_category', columns=None, values='debt', aggfunc=['count', 'sum', 'mean']).round(3)

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4288,400,0.093
операции с недвижимостью,10780,780,0.072
получение образования,3997,369,0.092
проведение свадьбы,2337,183,0.078


Вывод: Прослеживается повышенная доля должников в группах "операции с автомобилем" и "получение образования" по 9.3% в каждой. Более низкая доля должников присутствует в группах " проведение свадеб" и "опеации с недвижимостью" - 7.9% и 7.3% соответственно. Провслеживается незначительное влияние целей кредита (1-2%) на возврат в срок.

## Общий вывод

В данном исследовании реализованы следующие действия
1. Импортированы и изучены данные
2. Обработаны аномальные значения, пропуска, и дубликаты. Скорректированы типы данных.
3. Категоризирован уровень дохода и цели кредита.
4. Проведен анализ зависимости основных факторов на целевую переменную.

Анализ зависимости по семейному положению показал наличие повышенной доли должников среди неженатых заёмщиков и состоящих в гражданском браке. 

Прямой зависимости между количеством детей, а также уровнем дохода и возвратом кредита в срок не выявлено. Но вместе с тем, можно отметить наибольший процент должников (9.1%) среди наименее обеспеченной группы "Е". А наименьшая доля должников (6.0%) в группе "D" - второй по обеспеченности снизу.

Также прослеживается незначительное влияние целей кредита (1-2%) на возврат в срок. Повышенная доля должников в группах "операции с автомобилем" и "получение образования" по 9.3% в каждой. Более низкая доля должников присутствует в группах "проведение свадеб" и "опеации с недвижимостью" - 7.9% и 7.3% соответственно.